# 自动微分模块

In [1]:
import torch

In [2]:
def test1():#标量的梯度计算

    x = torch.tensor(10, requires_grad=True, dtype=torch.float32)
    
    f = x**2 + 2*x + 1
    f.backward()#自动计算梯度
    print(x.grad)#打印梯度

test1()


tensor(22.)


In [ ]:
def test2():#向量的梯度计算
    x = torch.tensor([1,2,3], requires_grad=True, dtype=torch.float32)
    f = 2*x**2 + 3*x + 4
    f.sum().backward()#自动微分时需要将向量变为标量
    print(x.grad)

test2()

tensor([ 7., 11., 15.])


In [5]:
def test3():# 多标量梯度计算
    x1 = torch.tensor(1.0, requires_grad=True)
    x2 = torch.tensor(2.0, requires_grad=True)

    y1 = x1**2 + x2**3 + x1*x2

    y1.backward()#默认只计算y1的梯度
    print(x1.grad)
    print(x2.grad)

test3()


tensor(4.)
tensor(13.)


In [7]:
def test4():# 多向量梯度计算
    x1 = torch.tensor([1.0,2.0], requires_grad=True)
    x2 = torch.tensor([3.0,4.0], requires_grad=True)

    y1 = x1**2 + x2**3 + x1*x2

    y1.sum().backward()#默认只计算y1的梯度
    print(x1.grad)
    print(x2.grad)

test4( )

tensor([5., 8.])
tensor([28., 50.])


# 控制梯度计算

In [3]:
def test5():
    x = torch.tensor(2.0, requires_grad=True)
    y = x**2

    #在不需要梯度计算的代码块中
    with torch.no_grad():
        z = y**3
    print(z.requires_grad)#False

    y.backward()#只计算y对x的梯度
    print(x.grad)

test5()

@torch.no_grad()#装饰器，表示该函数不需要梯度计算
def test6(x):
    return x**2 + 2*x + 1

y = test6(torch.tensor(2.0, requires_grad=True))
print(y.requires_grad)#False
    

False
tensor(4.)
False


# 累计梯度和梯度清零

In [10]:
def test7():# 累计梯度和梯度清零
    x = torch.tensor([5, 10, 15], requires_grad=True, dtype=torch.float32)
    for i in range(3):
        y = x**2
        y1 = y.mean()#计算均值的梯度
        y1.backward()
        print(x.grad)
        
test7()

tensor([ 3.3333,  6.6667, 10.0000])
tensor([ 6.6667, 13.3333, 20.0000])
tensor([10., 20., 30.])


# 梯度下降

In [13]:
def test8():
    # y = x**2
    x = torch.tensor(5, dtype=torch.float32, requires_grad=True )
    learning_rate = 0.1
    for i in range(10):
        y = x**2
        y.backward()
        with torch.no_grad():#不需要梯度计算
            x -= learning_rate * x.grad#梯度下降
            x.grad.zero_()#梯度清零
            print(f'epoch {i}: x={x}, y={y}')

test8()

epoch 0: x=4.0, y=25.0
epoch 1: x=3.200000047683716, y=16.0
epoch 2: x=2.559999942779541, y=10.24000072479248
epoch 3: x=2.047999858856201, y=6.553599834442139
epoch 4: x=1.6383998394012451, y=4.194303512573242
epoch 5: x=1.3107198476791382, y=2.684354066848755
epoch 6: x=1.0485758781433105, y=1.7179864645004272
epoch 7: x=0.8388606905937195, y=1.0995113849639893
epoch 8: x=0.6710885763168335, y=0.7036872506141663
epoch 9: x=0.5368708372116089, y=0.45035988092422485


# 梯度计算注意
- 设置为requires_grad = True的张量无法转换为numpy,需要先用detach函数将张量分离,再使用numpy函数

In [ ]:
def test9():
    # 设置为requires_grad = True的张量无法转换为numpy,需要先用detach函数将张量分离,再使用numpy函数
    x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
    '''
    detach生成的张量和原张量共享内存,但不需要梯度计算
    '''
    print(x.detach().numpy())#正确
test9()

[1. 2. 3.]
